<a href="https://colab.research.google.com/github/shashankbhagat/Multilabel-Image-Classification/blob/master/Multi_Label_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import re
%matplotlib inline

In [0]:
#Download csv from Google Drive
csv_link='https://drive.google.com/open?id=1fHX_qjo6zMZZyLHVLpdxtYnG528tU_U4'
temp, csv_id=csv_link.split('=')
downloaded=drive.CreateFile({'id':csv_id})
downloaded.GetContentFile('train.csv')


In [5]:
train=pd.read_csv('train.csv')
train.head()

,Id,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,N/A,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,tt0086425,"['Comedy', 'Drama']",0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0085549,"['Drama', 'Romance', 'Music']",0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,tt0086465,['Comedy'],0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0086567,"['Sci-Fi', 'Thriller']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,tt0086034,"['Action', 'Adventure', 'Thriller']",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
train.columns

Index(['Id', 'Genre', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'N/A', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [61]:
#Download images from google drive
pattern=re.compile('[a-zA-Z0-9]+[.]jpg')
local_img_path=os.path.expanduser(os.getcwd()+'/Images')
try:
    os.makedirs(local_img_path)
except: pass

print(local_img_path)
img_folder_link='https://drive.google.com/open?id=1zoWXcDzfjxeYbvOknbGU5WyM707yDGcP'
temp,img_folder_id=img_folder_link.split('=')
print(img_folder_id)
#downloaded=drive.CreateFile({'id':img_folder_id})
#downloaded.GetContentFile('Images')
fileList=drive.ListFile({'q':"'{}' in parents".format(str(img_folder_id))}).GetList()
for f in tqdm(fileList):
    #print(f['title'])
    if bool(re.match(pattern,f['title'])):
        try:
            fname=os.path.join(local_img_path,f['title'])
            downloaded=drive.CreateFile({'id':f['id']})
            downloaded.GetContentFile(fname)
        except: pass

/content/Images
1zoWXcDzfjxeYbvOknbGU5WyM707yDGcP



  0%|          | 0/7876 [00:00<?, ?it/s]

tt0096211.jpg



  0%|          | 1/7876 [00:10<22:56:33, 10.49s/it]

tt0096203.jpg


...

tt5022418.jpg



  0%|          | 16/7876 [00:45<2:19:32,  1.07s/it]

tt4944352.jpg



  0%|          | 17/7876 [00:45<1:51:10,  1.18it/s]

KeyboardInterrupt: ignored

In [53]:
import re
pattern=re.compile('[a-zA-Z0-9]+[.]jpg')
print(bool(re.match(pattern,'23sdf3dsfds45.jpg')))

True


In [27]:
#preprocessing the data
train_image=[]
for i in tqdm(range(train.shape[0])):
    img=image.load_img('Images/'+train['Id'][i]+'.jpg',target_size=(400,400,3))
    img=image.img_to_array(img)
    img=img/255
    train_image.append(img)
X=np.array(train_image)


  0%|          | 0/7254 [00:00<?, ?it/s]


FileNotFoundError: ignored

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
